In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 4}
----------
[[ 1377  5340]
 [    0 17377]]
----------
              precision    recall  f1-score   support

           0       1.00      0.21      0.34      6717
           1       0.76      1.00      0.87     17377

    accuracy                           0.78     24094
   macro avg       0.88      0.60      0.60     24094
weighted avg       0.83      0.78      0.72     24094

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.18300986, 0.21392775, 0.29022336, 0.32114089, 0.3296181 ,
        0.3341068 , 0.40441847, 0.44481075, 0.4153893 , 0.45328784,
        0.51761687, 0.50963759, 0.53506863, 0.53706324, 0.76894331,
        0.24384654, 0.29720592, 0.3266263 , 0.39444482, 0.42286885,
        0.47722363, 0.56498837, 0.60787439, 0.61734855, 0.67220306,
        0.704615  , 0.76794624, 0.77093697, 0.73755491, 0.77442753]),
 'std_fit_time': array([1.14684105e-02, 1.49607658e-03, 2.99239159e-03, 1.79513693e-02,
        2.04461813e-02, 4.98688221e-03, 2.49266624e-03, 5.96046448e-07,
        2.49218941e-03, 2.49266624e-03, 5.98299503e-03, 2.19419003e-02,
        1.44611597e-02, 2.49373913e-03, 2.19416618e-02, 1.44611597e-02,
        7.97891617e-03, 1.49655342e-03, 1.84503794e-02, 6.98196888e-03,
        4.98533249e-04, 2.94215679e-02, 5.48553467e-03, 9.97316837e-03,
        1.99365616e-03, 7.48050213e-03, 4.98723984e-03, 2.99143791e-03,
        4.45973873e-03, 2.14425325e-02]),
 'mean_scor